In [1]:
from pandas import set_option, DataFrame as DF, MultiIndex

from main import get_treasury_portfolio, get_treasury, get_historical_price_by_symbol

In [2]:
portfolio = await get_treasury_portfolio("0x1a9C8182C09F50C8318d769245beA52c32BE35BC")

In [3]:
uni_v2_treasury_vested = await get_treasury(portfolio)

In [4]:
set_option("display.precision", 6)
treasury_assets = DF(data=uni_v2_treasury_vested.assets, index=[asset["token_symbol"] for asset in uni_v2_treasury_vested.assets])
treasury_assets.drop(["token_name", "token_symbol"], axis=1, inplace=True)
treasury_assets.rename_axis("token_symbol", inplace=True)

treasury_assets

,token_address,balance
token_symbol,,
UNI,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,1.389651e+09
ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,3.880146e+00
ACE,0xec5483804e637d45cde22fa0869656b64b5ab1ab,2.096559e+00
DOTC,0xc0748cf90e231b8f21f75b7ad69732f19c76b751,1.633615e+00
SDOG,0x537edd52ebcb9f48ff2f8a28c51fcdb9d6a6e0d4,8.533554e-05
WJXN,0xca1262e77fb25c0a4112cfc9bad3ff54f617f2e6,1.317882e+01


In [5]:
treasury_assets.to_csv("uni_treasury_assets.csv")

In [40]:
historical_price_for_portfolio_assets = []
for symbol in treasury_assets.index.values:
    covalent_resp = await get_historical_price_by_symbol(
        symbol,
        "a_year_ago"
    )

    if not covalent_resp:
        continue

    indexes = MultiIndex.from_tuples(
        [
            (ts, address, symbol) for ts, address
            in zip(
                [ price["date"] for price in covalent_resp["prices"] ],
                [ price["contract_metadata"]["contract_address"] for price in covalent_resp["prices"] ]
            )
        ],
        names=["timestamp", "address", "symbol"]
    )

    historical_price_for_portfolio_assets.append(
        DF(
            data=covalent_resp["prices"],
            index=indexes
        )
        .drop(["contract_metadata", "date"], axis=1)
    )

historical_price_for_portfolio_assets[0]

,,,price
timestamp,address,symbol,
2022-05-11,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,6.150599
2022-05-10,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,6.159834
2022-05-09,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,6.834505
2022-05-08,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,7.463992
2022-05-07,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,7.294789
...,...,...,...
2021-05-15,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,38.813053
2021-05-14,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,40.232098
2021-05-13,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,37.721592


In [54]:
from math import log
for df in historical_price_for_portfolio_assets:
    address_level = df.index.get_level_values(1)
    address = address_level[0]
    symbol_level = df.index.get_level_values(2)
    symbol = symbol_level[0]
    df = df.reset_index()

    returns = []
    for i in range(1, len(df)):
        try:
            returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
        except Exception as e:
            print("misbehaving case: \n")
            print(f"\tsymbol: {symbol}\n\tindex: {i}\n\tcurr_price: {df.loc[i, 'price']}\n\tprev_price: {df.loc[i - 1, 'price']}")
            returns.append(None)
    returns.append(0)
    df["returns"] = returns

    df.to_csv(
        f"hist_prices/{str(symbol) + '_' + str(address)[0:6] + '_hist_price'}.csv"
    )

current:  6.150599
prev:  6.1598344 
---
current:  6.1598344
prev:  6.8345046 
---
current:  2360.7031
prev:  2371.859 
---
current:  2371.859
prev:  2524.4504 
---
current:  0.0
prev:  0.0 
---
current:  0.0
prev:  0.0 
---
misbehaving case: 

	symbol: ACE
	index: 27
	curr_price: 4.4383885e-08
	prev_price: 0.0
current:  1.2477759e-07
prev:  1.6466255e-07 
---
current:  1.6466255e-07
prev:  1.5141009e-07 
---
misbehaving case: 

	symbol: SDOG
	index: 219
	curr_price: 4.67809e-06
	prev_price: 0.0
misbehaving case: 

	symbol: SDOG
	index: 223
	curr_price: 5.6928975e-06
	prev_price: 0.0


/tmp/ipykernel_25559/4260102998.py:15: RuntimeWarning: invalid value encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
/tmp/ipykernel_25559/4260102998.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
/tmp/ipykernel_25559/4260102998.py:15: RuntimeWarning: invalid value encountered in double_scalars
  returns.append(log(df.loc[i - 1, "price"] / df.loc[i, "price"]))
